In [1]:
import torch
import torchvision
from google.colab import drive
drive.mount('/content/drive')
from PIL import Image
import torchvision.datasets as datasets
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt

Mounted at /content/drive


In [120]:
import numpy

def binarize_image(image_tensor):
    image_tensor[image_tensor > 0.5] = 1
    image_tensor[image_tensor <= 0.5] = 0
    return image_tensor

def get_labels_probabilities(labels_occurences):
  uniq, counts = labels_occurences.unique(return_counts=True)
  #print("uniq, counts:", uniq, counts)
  return torch.div(counts, labels_occurences.shape[0])

def generate_probabilities_matrix(train_data_tensor, labels_training, labels_probabilities, num_classes):
  #print("***************Train data tensor:", train_data_tensor, train_data_tensor.shape)
  #print("***************Train data tensor:", train_data_tensor[:, 0], train_data_tensor[:, 0].shape)
  complete_matrix = torch.zeros(num_classes, train_data_tensor.shape[0])
  #print(complete_matrix, complete_matrix.shape)
  general_probabilities_matrix = []
  for k in range(0, num_classes):
    condition = labels_training == k
    indices = condition.nonzero().reshape(-1)
    #print("Indices of k=", k, indices)
    k_type_observations = train_data_tensor[:, indices]
    #print(k_type_observations, k_type_observations.shape)
    #print(k_type_observations.shape[0])
    k_pixels_probabilities = []
    for pixel_pos in range(0, k_type_observations.shape[0]):
        #La idea es agarrar todos los pixeles que estan en la posicion pixel en las 60 imagenes para la k clase
        pixels_in_position = k_type_observations[pixel_pos, :]
        #print(pixels_in_position)
        #print(pixels_in_position.shape)
        #Los que estan desactivados
        condition2 = pixels_in_position == 0.
        occurences_of_zero = condition2.nonzero().reshape(-1).shape[0]
        #print("Pixels in position", pixel_pos, pixels_in_position.shape)
        #print("Ocurrences of zero", occurences_of_zero)
        #print("Probability of zero", occurences_of_zero / pixels_in_position.shape[0])
        k_pixels_probabilities.append(occurences_of_zero / pixels_in_position.shape[0])
    tensor_k_pixels_probabilities = torch.tensor(k_pixels_probabilities)
    complete_matrix[k].add_(tensor_k_pixels_probabilities)
    #print(tensor_k_pixels_probabilities)
  #print(complete_matrix, complete_matrix.shape)
  return complete_matrix

def get_probability_pixel_by_class(pixel, pixel_index, p_m_pix_val_given_k, num_class):
  #print(p_m_pix_val_given_k)
  matrix_given_by_pixel = p_m_pix_val_given_k[int(pixel)]
  #print("Matrix given by the pixel itself", matrix_given_by_pixel)
  pixel_probability_in_class = matrix_given_by_pixel[num_class, pixel_index]
  return pixel_probability_in_class

def test_model(input_torch, p_m_pix_val_given_k, p_t_tensor, num_classes = 10):
    #assumes that the input comes in a row
    #TODO IMPLEMENT
    k_pixels_probabilities = []
    print(input_torch)
    for k in range(0, 1):
      print("************k******************", k)
      mult = 1
      for pixel_index in range(0, input_torch.shape[0]):
        #print(input_torch[pixel_index].item())
        probability_class_for_pixel = get_probability_pixel_by_class(input_torch[pixel_index].item(), pixel_index, p_m_pix_val_given_k, k).item()
        #Suavizado laplaciano
        if(probability_class_for_pixel == 0.0):
          probability_class_for_pixel = 0.0001
        print("Probability",probability_class_for_pixel)
        mult *= probability_class_for_pixel * numpy.float32(1)
        #mult = round(mult,8)
        print("Mult", mult)
      k_pixels_probabilities.append(mult)
    print(k_pixels_probabilities)
    return (0, 1) 
    #return (predicted_label, scores_classes)

#def test_model_batch(test_set, labels, p_m_pix_val_given_k, p_t_tensor):
    #TODO IMPLEMENT

def train_model(train_data_tensor, labels_training, num_classes = 10):
    #TODO IMPLEMENT
    #Primero ocupo sacar la probabilidad de cada feature - p(ti)
    labels_probabilities = get_labels_probabilities(labels_training)
    #print(labels_probabilities)
    matrix_probabilities_0_given_k = generate_probabilities_matrix(train_data_tensor, labels_training, labels_probabilities, num_classes)
    matrix_probabilities_1_given_k = 1 - matrix_probabilities_0_given_k
    p_m_pix_val_given_k = [matrix_probabilities_0_given_k, matrix_probabilities_1_given_k]
    return (p_m_pix_val_given_k, labels_probabilities)



def load_dataset(path = "/content/drive/MyDrive/Colab Notebooks/mnist_dataset/train"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    #print(dataset)
    list_images = dataset.imgs
    #print(list_images)
    train_data_tensor  = None    
    labels_training = []
    first_tensor = True
    #list_images_training =  set(data_labeled.train_ds.x.items)
    #print(list_images)

    ### PREGUNTAR PORQUE EL PROFE EMPEZO EN 1 EL RANGE
    for i in range(0, len(list_images)):
        pair_path_label = list_images[i]        
        image = Image.open(pair_path_label[0]) 
        x_tensor = TF.to_tensor(image).squeeze()
        x_tensor_bin = binarize_image(x_tensor)       
        #plt.figure()
        #plt.imshow(x_tensor_bin)
        x_tensor_bin_plain = x_tensor_bin.view(x_tensor_bin.shape[0] * x_tensor_bin.shape[1], -1)

        #print("x  tensor ", x_tensor.shape)
        #print("x tensor binario ", x_tensor_bin.shape)
        #print("x tensor aplanado", x_tensor_bin_plain.shape)

        #test dataset case        
        #if("train" in pair_path_label[0]):
        labels_training += [pair_path_label[1]]
        #print(pair_path_label)
        
        if(first_tensor):
            first_tensor = False
            train_data_tensor = x_tensor_bin_plain
        else:
            train_data_tensor = torch.cat((train_data_tensor, x_tensor_bin_plain), 1)

        #print("Train datatensor - matriz con los vectores aplanados", train_data_tensor.shape)    
    return (train_data_tensor, torch.tensor(labels_training))       

#(train_data_tensor, labels_training) = load_dataset()
#print("train dimensions ", train_data_tensor.shape)
#print("train labels ", len(labels_training))

#train model by calculating the prior probabilities
# El p_m_pix_val_given_k es la matriz de la probabilidad de 0 o 1
# El p_t_tensor son las probabilidades de las k clases
#(p_m_pix_val_given_k, p_t_tensor) = train_model(train_data_tensor, labels_training)
#print("p_m_pix_val_given_k size ", p_m_pix_val_given_k.shape)
#OJO ESTE TEST MODEL ES SOLO PARA 1 IMAGEN
#(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor)
#print("predicted_label ", predicted_label)
#print("real label ", labels_training[500])
#acc = test_model_batch(train_data_tensor, labels_training, p_m_pix_val_given_k, p_t_tensor)
#print("Model accuracy ", acc)
    
(train_data_tensor, labels_tensor) = load_dataset(path = "/content/drive/MyDrive/Colab Notebooks/mnist_dataset/train")

# print("train_data_tensor shape ", train_data_tensor.shape)
# print("train_data_tensor", train_data_tensor)
# print("labels_tensor shape", labels_tensor.shape)
# print("labels_tensor", labels_tensor)

p_m_pix_val_given_k , p_t_tensor = train_model(train_data_tensor, labels_tensor)

#Predecir el lable de una observacion, la observacion #500
(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor, 10)
#print("predicted_label ", predicted_label)
#print("real label ", labels_training[500])
               
    
    

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [58]:
t = torch.tensor([[4, 3, 6], [5, 2, 3], [1, 7, 8]])
idc = torch.tensor([ 0,  2])
print(t[idc])

gg = torch.tensor([0.7767857142857143, 0.7857142857142857, 0.7130102040816326, 0.8826530612244898, 0.8494897959183674, 0.8596938775510204, 0.7576530612244898, 0.8456632653061225, 0.8507653061224489, 0.8724489795918368, 0.8494897959183674, 0.8341836734693877, 0.8533163265306123, 0.8099489795918368, 0.8660714285714286, 0.8571428571428571, 0.8303571428571429, 0.8380102040816326, 0.8877551020408163, 0.7818877551020408, 0.8239795918367347, 0.7678571428571429, 0.7538265306122449, 0.8520408163265306, 0.8418367346938775, 0.8125, 0.8698979591836735, 0.7971938775510204, 0.8329081632653061, 0.8571428571428571, 0.8482142857142857, 0.860969387755102, 0.7423469387755102, 0.7614795918367347, 0.8469387755102041, 0.8341836734693877, 0.7551020408163265, 0.8214285714285714, 0.7959183673469388, 0.8418367346938775, 0.8227040816326531, 0.8201530612244898, 0.7474489795918368, 0.7704081632653061, 0.8533163265306123, 0.7691326530612245, 0.8073979591836735, 0.8431122448979592, 0.8405612244897959, 0.8086734693877551, 0.8443877551020408, 0.8839285714285714, 0.8647959183673469, 0.7066326530612245, 0.860969387755102, 0.8163265306122449, 0.8456632653061225, 0.8290816326530612, 0.9158163265306123, 0.8405612244897959])
print(gg.shape)

tensor([[4, 3, 6],
        [1, 7, 8]])
torch.Size([60])
